## 0. Introduction

Ce projet est réalisé par Zakarya El Mimouni et Yassine Boukhateb étudiants en deuxième année du cycle ingénieur de l'ENSAE Paris dans le cadre du cours de Python pour la DataScience.

**Importation des modules et packages**

In [62]:
import requests
import json
import time
import pandas as pd
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

## I. Récupération des données

On récupère les données de 10 000 films grâce à l'API TMDB

In [6]:
# Votre clé d'API TMDb
api_key = '28c6630049f5d468217e4f34963c03a6'

# URL de base de l'API TMDb pour les films populaires
base_url_popular = 'https://api.themoviedb.org/3/movie/popular'

# Nombre total de films à récupérer
total_movies = 10000

# Nombre de films à récupérer par requête
movies_per_request = 20

# Champs de base pour les films
fields = 'id'  # Ajoutez les champs que vous souhaitez récupérer ici

# Liste pour stocker tous les films avec leurs détails
all_movies_details = []

# Nombre de requêtes nécessaires pour obtenir le nombre total de films
num_requests = total_movies // movies_per_request

# Effectuer les requêtes pour récupérer les films
for page in range(1, num_requests + 1):
    params = {
        'api_key': api_key,
        'page': page,
        'fields': fields
    }
    response = requests.get(base_url_popular, params=params)#, timeout=10)  # Définir un délai d'attente de 10 secondes

    if response.status_code == 200:
        movies_data = response.json()
        movies_reco = movies_data.get('results', [])

            # Pour chaque film, récupérer les détails supplémentaires
        for movie in movies_reco:
            movie_id = movie['id']
            movie_details_url = f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}&append_to_response=credits,keywords'
            movie_details_response = requests.get(movie_details_url)
            if movie_details_response.status_code == 200:
                movie_details = movie_details_response.json()
                all_movies_details.append(movie_details)
                print(f"Film {movie['title']} récupéré avec détails supplémentaires.")
            else:
                print(f"Erreur lors de la récupération des détails pour {movie['title']}. Statut : {movie_details_response.status_code}")
        print(f"Page {page} récupérée. Total de films récupérés : {len(all_movies_details)}")

    else:
        print(f"Erreur lors de la requête pour la page {page}. Statut : {response.status_code}")


    if len(all_movies_details) >= total_movies:
        break

# Enregistrer les données complètes dans un fichier JSON
with open('movies_data_complete.json', 'w', encoding='utf-8') as f:
    json.dump(all_movies_details, f, ensure_ascii=False, indent=4)

print("Récupération des films avec détails complets terminée. Les données ont été enregistrées dans movies_data_complete.json.")

Film Killers of the Flower Moon récupéré avec détails supplémentaires.
Film Robot Apocalypse récupéré avec détails supplémentaires.
Film Freelance récupéré avec détails supplémentaires.
Film Wonka récupéré avec détails supplémentaires.
Film Trolls Band Together récupéré avec détails supplémentaires.
Film Leave the World Behind récupéré avec détails supplémentaires.
Film Leo récupéré avec détails supplémentaires.
Film Good Boy récupéré avec détails supplémentaires.
Film A Man of Reason récupéré avec détails supplémentaires.
Film Oppenheimer récupéré avec détails supplémentaires.
Film Five Nights at Freddy's récupéré avec détails supplémentaires.
Film The Creator récupéré avec détails supplémentaires.
Film The Advent Calendar récupéré avec détails supplémentaires.
Film Rumble Through the Dark récupéré avec détails supplémentaires.
Film Fast X récupéré avec détails supplémentaires.
Film Fast Charlie récupéré avec détails supplémentaires.
Film Merry Little Batman récupéré avec détails supp

On récupère les données qui nous seront utile pour la création de l'interface de l'application.

In [21]:
movies_data=pd.read_json('movies_data_complete.json')
selected_columns=['id','title','original_title','overview','release_date','adult','popularity','vote_average','vote_count','poster_path','runtime','genres','production_companies','production_countries','spoken_languages','credits','keywords']
movies_data=movies_data=movies_data[selected_columns]
movies_data.head()

,id,title,original_title,overview,release_date,adult,popularity,vote_average,vote_count,poster_path,runtime,genres,production_companies,production_countries,spoken_languages,credits,keywords
0,466420,Killers of the Flower Moon,Killers of the Flower Moon,When oil is discovered in 1920s Oklahoma under...,2023-10-18,False,1323.647,7.617,1445,/dB6Krk806zeqd0YNp2ngQ9zXteH.jpg,206,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...","[{'id': 194232, 'logo_path': '/oE7H93u8sy5vvW5...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'english_name': 'English', 'iso_639_1': 'en'...","{'cast': [{'adult': False, 'gender': 2, 'id': ...","{'keywords': [{'id': 1157, 'name': 'husband wi..."
1,854648,Robot Apocalypse,Robot Apocalypse,An expert hacker is targeted by a sentient AI ...,2021-07-26,False,692.968,6.048,62,/zpbgktIR7GkOS83PBAzLlzLSQ5W.jpg,87,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...","[{'id': 1311, 'logo_path': '/ic2bTizdzRLDVzAvN...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'english_name': 'English', 'iso_639_1': 'en'...","{'cast': [{'adult': False, 'gender': 2, 'id': ...",{'keywords': []}
2,897087,Freelance,Freelance,An ex-special forces operative takes a job to ...,2023-01-05,False,1243.624,6.535,317,/7Bd4EUOqQDKZXA6Od5gkfzRNb0.jpg,108,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...","[{'id': 89171, 'logo_path': '/c3ttVfx0itQzk2vO...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'english_name': 'English', 'iso_639_1': 'en'...","{'cast': [{'adult': False, 'gender': 2, 'id': ...","{'keywords': [{'id': 6285, 'name': 'coup d'eta..."
3,787699,Wonka,Wonka,Willy Wonka – chock-full of ideas and determin...,2023-12-06,False,1119.142,7.317,278,/qhb1qOilapbapxWQn9jtRCMwXJF.jpg,117,"[{'id': 35, 'name': 'Comedy'}, {'id': 10751, '...","[{'id': 174, 'logo_path': '/IuAlhI9eVC9Z8UQWOI...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...","[{'english_name': 'English', 'iso_639_1': 'en'...","{'cast': [{'adult': False, 'gender': 2, 'id': ...","{'keywords': [{'id': 715, 'name': 'chocolate'}..."
4,901362,Trolls Band Together,Trolls Band Together,"When Branch's brother, Floyd, is kidnapped for...",2023-10-12,False,968.800,7.169,370,/bkpPTZUdq31UGDovmszsg2CchiI.jpg,92,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...","[{'id': 521, 'logo_path': '/kP7t6RwGz2AvvTkvnI...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'english_name': 'English', 'iso_639_1': 'en'...","{'cast': [{'adult': False, 'gender': 1, 'id': ...","{'keywords': [{'id': 3490, 'name': 'pop star'}..."


On contrôle qu'il n'y pas de variables avec des informations manquantes. On fait le choix d'enlever le film dès lors qu'il manque une info c'est assez restrictif A FAIRE PLUS TARD

In [32]:
movies_data.isnull().sum()


id                      0
title                   0
original_title          0
overview                0
release_date            0
adult                   0
popularity              0
vote_average            0
vote_count              0
poster_path             0
runtime                 0
genres                  0
production_companies    0
production_countries    0
spoken_languages        0
credits                 0
keywords                0
dtype: int64

In [33]:
movies_data.dropna(axis=0,inplace=True)

Pour notre modèle on conserve que les films dont les features ci dessous existent ça sera plus crédible

In [44]:
movies_reco=movies_data[["id","title","overview","genres","keywords","credits"]]

On nettoie les colonnes genres, keywords, credits

In [45]:
movies_reco.loc[0].genres

[{'id': 80, 'name': 'Crime'},
 {'id': 18, 'name': 'Drama'},
 {'id': 36, 'name': 'History'}]

In [46]:
def convert(text):
    L = []
    for i in text:
        L.append(i["name"]) 
    return L 
movies_reco['genres'] = movies_reco['genres'].apply(convert)
movies_reco.head()

/var/folders/8w/8q5hgk2d2wnfsvdq9brjz71h0000gn/T/ipykernel_4060/202457164.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_reco['genres'] = movies_reco['genres'].apply(convert)


,id,title,overview,genres,keywords,credits
0,466420,Killers of the Flower Moon,When oil is discovered in 1920s Oklahoma under...,"[Crime, Drama, History]","{'keywords': [{'id': 1157, 'name': 'husband wi...","{'cast': [{'adult': False, 'gender': 2, 'id': ..."
1,854648,Robot Apocalypse,An expert hacker is targeted by a sentient AI ...,"[Science Fiction, Action]",{'keywords': []},"{'cast': [{'adult': False, 'gender': 2, 'id': ..."
2,897087,Freelance,An ex-special forces operative takes a job to ...,"[Action, Comedy]","{'keywords': [{'id': 6285, 'name': 'coup d'eta...","{'cast': [{'adult': False, 'gender': 2, 'id': ..."
3,787699,Wonka,Willy Wonka – chock-full of ideas and determin...,"[Comedy, Family, Fantasy]","{'keywords': [{'id': 715, 'name': 'chocolate'}...","{'cast': [{'adult': False, 'gender': 2, 'id': ..."
4,901362,Trolls Band Together,"When Branch's brother, Floyd, is kidnapped for...","[Animation, Family, Music, Fantasy, Comedy]","{'keywords': [{'id': 3490, 'name': 'pop star'}...","{'cast': [{'adult': False, 'gender': 1, 'id': ..."


In [47]:
movies_reco['keywords'] = movies_reco['keywords'].apply(lambda x: x['keywords'] if isinstance(x, dict) and 'keywords' in x else None) # regarder Robot Apocalypse Keywords
movies_reco

/var/folders/8w/8q5hgk2d2wnfsvdq9brjz71h0000gn/T/ipykernel_4060/293773367.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_reco['keywords'] = movies_reco['keywords'].apply(lambda x: x['keywords'] if isinstance(x, dict) and 'keywords' in x else None) # regarder Robot Apocalypse Keywords


,id,title,overview,genres,keywords,credits
0,466420,Killers of the Flower Moon,When oil is discovered in 1920s Oklahoma under...,"[Crime, Drama, History]","[{'id': 1157, 'name': 'husband wife relationsh...","{'cast': [{'adult': False, 'gender': 2, 'id': ..."
1,854648,Robot Apocalypse,An expert hacker is targeted by a sentient AI ...,"[Science Fiction, Action]",[],"{'cast': [{'adult': False, 'gender': 2, 'id': ..."
2,897087,Freelance,An ex-special forces operative takes a job to ...,"[Action, Comedy]","[{'id': 6285, 'name': 'coup d'etat'}, {'id': 7...","{'cast': [{'adult': False, 'gender': 2, 'id': ..."
3,787699,Wonka,Willy Wonka – chock-full of ideas and determin...,"[Comedy, Family, Fantasy]","[{'id': 715, 'name': 'chocolate'}, {'id': 4344...","{'cast': [{'adult': False, 'gender': 2, 'id': ..."
4,901362,Trolls Band Together,"When Branch's brother, Floyd, is kidnapped for...","[Animation, Family, Music, Fantasy, Comedy]","[{'id': 3490, 'name': 'pop star'}, {'id': 5301...","{'cast': [{'adult': False, 'gender': 1, 'id': ..."
5,726209,Leave the World Behind,A family's getaway to a luxurious rental home ...,"[Drama, Mystery, Thriller, Science Fiction]","[{'id': 818, 'name': 'based on novel or book'}...","{'cast': [{'adult': False, 'gender': 1, 'id': ..."
6,1075794,Leo,Jaded 74-year-old lizard Leo has been stuck in...,"[Animation, Comedy, Family]","[{'id': 1517, 'name': 'classroom'}, {'id': 434...","{'cast': [{'adult': False, 'gender': 2, 'id': ..."
7,983507,Good Boy,"Christian - a millionaire heir, meets Sigrid -...","[Horror, Thriller]","[{'id': 15162, 'name': 'dog'}, {'id': 220712, ...","{'cast': [{'adult': False, 'gender': 2, 'id': ..."
8,656156,A Man of Reason,After 10 years of incarceration on behalf of h...,"[Action, Thriller, Drama]","[{'id': 10726, 'name': 'gang'}, {'id': 298636,...","{'cast': [{'adult': False, 'gender': 2, 'id': ..."
9,872585,Oppenheimer,The story of J. Robert Oppenheimer's role in t...,"[Drama, History]","[{'id': 1157, 'name': 'husband wife relationsh...","{'cast': [{'adult': False, 'gender': 2, 'id': ..."


In [50]:
movies_reco = movies_reco[movies_reco['keywords'].apply(lambda x: isinstance(x, list) and len(x) > 0)]
movies_reco["keywords"]=movies_reco["keywords"].apply(convert)
movies_reco

/var/folders/8w/8q5hgk2d2wnfsvdq9brjz71h0000gn/T/ipykernel_4060/1419206400.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_reco["keywords"]=movies_reco["keywords"].apply(convert)


,id,title,overview,genres,keywords,credits
0,466420,Killers of the Flower Moon,When oil is discovered in 1920s Oklahoma under...,"[Crime, Drama, History]","[husband wife relationship, based on novel or ...","{'cast': [{'adult': False, 'gender': 2, 'id': ..."
2,897087,Freelance,An ex-special forces operative takes a job to ...,"[Action, Comedy]","[coup d'etat, dictatorship, political assassin...","{'cast': [{'adult': False, 'gender': 2, 'id': ..."
3,787699,Wonka,Willy Wonka – chock-full of ideas and determin...,"[Comedy, Family, Fantasy]","[chocolate, musical, prequel]","{'cast': [{'adult': False, 'gender': 2, 'id': ..."
4,901362,Trolls Band Together,"When Branch's brother, Floyd, is kidnapped for...","[Animation, Family, Music, Fantasy, Comedy]","[pop star, brother, musical, sequel, based on ...","{'cast': [{'adult': False, 'gender': 1, 'id': ..."
5,726209,Leave the World Behind,A family's getaway to a luxurious rental home ...,"[Drama, Mystery, Thriller, Science Fiction]","[based on novel or book, crisis, vacation]","{'cast': [{'adult': False, 'gender': 1, 'id': ..."
6,1075794,Leo,Jaded 74-year-old lizard Leo has been stuck in...,"[Animation, Comedy, Family]","[classroom, musical, bucket list]","{'cast': [{'adult': False, 'gender': 2, 'id': ..."
7,983507,Good Boy,"Christian - a millionaire heir, meets Sigrid -...","[Horror, Thriller]","[dog, costume horror]","{'cast': [{'adult': False, 'gender': 2, 'id': ..."
8,656156,A Man of Reason,After 10 years of incarceration on behalf of h...,"[Action, Thriller, Drama]","[gang, underworld]","{'cast': [{'adult': False, 'gender': 2, 'id': ..."
9,872585,Oppenheimer,The story of J. Robert Oppenheimer's role in t...,"[Drama, History]","[husband wife relationship, based on novel or ...","{'cast': [{'adult': False, 'gender': 2, 'id': ..."
10,507089,Five Nights at Freddy's,"Recently fired and desperate for work, a troub...","[Horror, Mystery]","[night shift, child murder, restaurant, custod...","{'cast': [{'adult': False, 'gender': 2, 'id': ..."


In [53]:
movies_reco['cast'] = movies_reco['credits'].apply(lambda x: x.get('cast') if isinstance(x, dict) else None)
movies_reco['crew'] = movies_reco['credits'].apply(lambda x: x.get('crew') if isinstance(x, dict) else None)
movies_reco.drop("credits", axis=1,inplace=True)
movies_reco.reset_index(drop=True,inplace=True)
movies_reco.head()

,id,title,overview,genres,keywords,cast,crew
0,466420,Killers of the Flower Moon,When oil is discovered in 1920s Oklahoma under...,"[Crime, Drama, History]","[husband wife relationship, based on novel or ...","[{'adult': False, 'gender': 2, 'id': 6193, 'kn...","[{'adult': False, 'gender': 2, 'id': 27, 'know..."
1,897087,Freelance,An ex-special forces operative takes a job to ...,"[Action, Comedy]","[coup d'etat, dictatorship, political assassin...","[{'adult': False, 'gender': 2, 'id': 56446, 'k...","[{'adult': True, 'gender': 2, 'id': 997, 'know..."
2,787699,Wonka,Willy Wonka – chock-full of ideas and determin...,"[Comedy, Family, Fantasy]","[chocolate, musical, prequel]","[{'adult': False, 'gender': 2, 'id': 1190668, ...","[{'adult': False, 'gender': 2, 'id': 1299, 'kn..."
3,901362,Trolls Band Together,"When Branch's brother, Floyd, is kidnapped for...","[Animation, Family, Music, Fantasy, Comedy]","[pop star, brother, musical, sequel, based on ...","[{'adult': False, 'gender': 1, 'id': 84223, 'k...","[{'adult': False, 'gender': 2, 'id': 10572, 'k..."
4,726209,Leave the World Behind,A family's getaway to a luxurious rental home ...,"[Drama, Mystery, Thriller, Science Fiction]","[based on novel or book, crisis, vacation]","[{'adult': False, 'gender': 1, 'id': 1204, 'kn...","[{'adult': False, 'gender': 0, 'id': 2447, 'kn..."


In [56]:
def convert3(text):
    L = []
    counter = 0
    for i in text:
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L 
movies_reco["cast"]=movies_reco["cast"].apply(convert3)
movies_reco.head()

,id,title,overview,genres,keywords,cast,crew
0,466420,Killers of the Flower Moon,When oil is discovered in 1920s Oklahoma under...,"[Crime, Drama, History]","[husband wife relationship, based on novel or ...","[Leonardo DiCaprio, Lily Gladstone, Robert De ...","[{'adult': False, 'gender': 2, 'id': 27, 'know..."
1,897087,Freelance,An ex-special forces operative takes a job to ...,"[Action, Comedy]","[coup d'etat, dictatorship, political assassin...","[John Cena, Alison Brie, Juan Pablo Raba]","[{'adult': True, 'gender': 2, 'id': 997, 'know..."
2,787699,Wonka,Willy Wonka – chock-full of ideas and determin...,"[Comedy, Family, Fantasy]","[chocolate, musical, prequel]","[Timothée Chalamet, Calah Lane, Keegan-Michael...","[{'adult': False, 'gender': 2, 'id': 1299, 'kn..."
3,901362,Trolls Band Together,"When Branch's brother, Floyd, is kidnapped for...","[Animation, Family, Music, Fantasy, Comedy]","[pop star, brother, musical, sequel, based on ...","[Anna Kendrick, Justin Timberlake, Camila Cabe...","[{'adult': False, 'gender': 2, 'id': 10572, 'k..."
4,726209,Leave the World Behind,A family's getaway to a luxurious rental home ...,"[Drama, Mystery, Thriller, Science Fiction]","[based on novel or book, crisis, vacation]","[Julia Roberts, Mahershala Ali, Ethan Hawke]","[{'adult': False, 'gender': 0, 'id': 2447, 'kn..."


In [57]:
def fetch_director(text):
    L = []
    for i in text:
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L 
movies_reco["crew"]=movies_reco["crew"].apply(fetch_director)
movies_reco.head()

/var/folders/8w/8q5hgk2d2wnfsvdq9brjz71h0000gn/T/ipykernel_4060/3728902092.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_reco["crew"]=movies_reco["crew"].apply(fetch_director)


,id,title,overview,genres,keywords,cast,crew
0,466420,Killers of the Flower Moon,When oil is discovered in 1920s Oklahoma under...,"[Crime, Drama, History]","[husband wife relationship, based on novel or ...","[Leonardo DiCaprio, Lily Gladstone, Robert De ...",[Martin Scorsese]
1,897087,Freelance,An ex-special forces operative takes a job to ...,"[Action, Comedy]","[coup d'etat, dictatorship, political assassin...","[John Cena, Alison Brie, Juan Pablo Raba]",[Pierre Morel]
2,787699,Wonka,Willy Wonka – chock-full of ideas and determin...,"[Comedy, Family, Fantasy]","[chocolate, musical, prequel]","[Timothée Chalamet, Calah Lane, Keegan-Michael...",[Paul King]
3,901362,Trolls Band Together,"When Branch's brother, Floyd, is kidnapped for...","[Animation, Family, Music, Fantasy, Comedy]","[pop star, brother, musical, sequel, based on ...","[Anna Kendrick, Justin Timberlake, Camila Cabe...",[Walt Dohrn]
4,726209,Leave the World Behind,A family's getaway to a luxurious rental home ...,"[Drama, Mystery, Thriller, Science Fiction]","[based on novel or book, crisis, vacation]","[Julia Roberts, Mahershala Ali, Ethan Hawke]",[Sam Esmail]


Ainsi, les colonnes genres, keywords, cast et crew sont exploitables.

In [58]:
movies_reco['overview'] = movies_reco['overview'].apply(lambda x:x.split())
movies_reco

/var/folders/8w/8q5hgk2d2wnfsvdq9brjz71h0000gn/T/ipykernel_4060/3778759656.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_reco['overview'] = movies_reco['overview'].apply(lambda x:x.split())


,id,title,overview,genres,keywords,cast,crew
0,466420,Killers of the Flower Moon,"[When, oil, is, discovered, in, 1920s, Oklahom...","[Crime, Drama, History]","[husband wife relationship, based on novel or ...","[Leonardo DiCaprio, Lily Gladstone, Robert De ...",[Martin Scorsese]
1,897087,Freelance,"[An, ex-special, forces, operative, takes, a, ...","[Action, Comedy]","[coup d'etat, dictatorship, political assassin...","[John Cena, Alison Brie, Juan Pablo Raba]",[Pierre Morel]
2,787699,Wonka,"[Willy, Wonka, –, chock-full, of, ideas, and, ...","[Comedy, Family, Fantasy]","[chocolate, musical, prequel]","[Timothée Chalamet, Calah Lane, Keegan-Michael...",[Paul King]
3,901362,Trolls Band Together,"[When, Branch's, brother,, Floyd,, is, kidnapp...","[Animation, Family, Music, Fantasy, Comedy]","[pop star, brother, musical, sequel, based on ...","[Anna Kendrick, Justin Timberlake, Camila Cabe...",[Walt Dohrn]
4,726209,Leave the World Behind,"[A, family's, getaway, to, a, luxurious, renta...","[Drama, Mystery, Thriller, Science Fiction]","[based on novel or book, crisis, vacation]","[Julia Roberts, Mahershala Ali, Ethan Hawke]",[Sam Esmail]
5,1075794,Leo,"[Jaded, 74-year-old, lizard, Leo, has, been, s...","[Animation, Comedy, Family]","[classroom, musical, bucket list]","[Adam Sandler, Bill Burr, Cecily Strong]",[Robert Smigel]
6,983507,Good Boy,"[Christian, -, a, millionaire, heir,, meets, S...","[Horror, Thriller]","[dog, costume horror]","[Gard Fartein Løkke Goli, Katrine Lovise Øpsta...",[Viljar Bøe]
7,656156,A Man of Reason,"[After, 10, years, of, incarceration, on, beha...","[Action, Thriller, Drama]","[gang, underworld]","[Jung Woo-sung, Kim Nam-gil, Park Sung-woong]",[Jung Woo-sung]
8,872585,Oppenheimer,"[The, story, of, J., Robert, Oppenheimer's, ro...","[Drama, History]","[husband wife relationship, based on novel or ...","[Cillian Murphy, Emily Blunt, Matt Damon]",[Christopher Nolan]
9,507089,Five Nights at Freddy's,"[Recently, fired, and, desperate, for, work,, ...","[Horror, Mystery]","[night shift, child murder, restaurant, custod...","[Josh Hutcherson, Piper Rubio, Elizabeth Lail]",[Emma Tammi]


In [59]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1
movies_reco['cast'] = movies_reco['cast'].apply(collapse)
movies_reco['crew'] = movies_reco['crew'].apply(collapse)
movies_reco['genres'] = movies_reco['genres'].apply(collapse)
movies_reco['keywords'] = movies_reco['keywords'].apply(collapse)
movies_reco.head()

/var/folders/8w/8q5hgk2d2wnfsvdq9brjz71h0000gn/T/ipykernel_4060/813522039.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_reco['cast'] = movies_reco['cast'].apply(collapse)
/var/folders/8w/8q5hgk2d2wnfsvdq9brjz71h0000gn/T/ipykernel_4060/813522039.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_reco['crew'] = movies_reco['crew'].apply(collapse)
/var/folders/8w/8q5hgk2d2wnfsvdq9brjz71h0000gn/T/ipykernel_4060/813522039.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy o

,id,title,overview,genres,keywords,cast,crew
0,466420,Killers of the Flower Moon,"[When, oil, is, discovered, in, 1920s, Oklahom...","[Crime, Drama, History]","[husbandwiferelationship, basedonnovelorbook, ...","[LeonardoDiCaprio, LilyGladstone, RobertDeNiro]",[MartinScorsese]
1,897087,Freelance,"[An, ex-special, forces, operative, takes, a, ...","[Action, Comedy]","[coupd'etat, dictatorship, politicalassassinat...","[JohnCena, AlisonBrie, JuanPabloRaba]",[PierreMorel]
2,787699,Wonka,"[Willy, Wonka, –, chock-full, of, ideas, and, ...","[Comedy, Family, Fantasy]","[chocolate, musical, prequel]","[TimothéeChalamet, CalahLane, Keegan-MichaelKey]",[PaulKing]
3,901362,Trolls Band Together,"[When, Branch's, brother,, Floyd,, is, kidnapp...","[Animation, Family, Music, Fantasy, Comedy]","[popstar, brother, musical, sequel, basedontoy...","[AnnaKendrick, JustinTimberlake, CamilaCabello]",[WaltDohrn]
4,726209,Leave the World Behind,"[A, family's, getaway, to, a, luxurious, renta...","[Drama, Mystery, Thriller, ScienceFiction]","[basedonnovelorbook, crisis, vacation]","[JuliaRoberts, MahershalaAli, EthanHawke]",[SamEsmail]


In [69]:
movies_reco['label'] = movies_reco['overview'] + movies_reco['genres'] + movies_reco['keywords'] + movies_reco['cast'] + movies_reco['crew']
data_model=movies_reco.drop(columns=['overview','genres','keywords','cast','crew'])
data_model['label'] = data_model['label'].apply(lambda x: " ".join(x))
data_model.head()

/var/folders/8w/8q5hgk2d2wnfsvdq9brjz71h0000gn/T/ipykernel_4060/3682922652.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_reco['label'] = movies_reco['overview'] + movies_reco['genres'] + movies_reco['keywords'] + movies_reco['cast'] + movies_reco['crew']


,id,title,label
0,466420,Killers of the Flower Moon,When oil is discovered in 1920s Oklahoma under...
1,897087,Freelance,An ex-special forces operative takes a job to ...
2,787699,Wonka,Willy Wonka – chock-full of ideas and determin...
3,901362,Trolls Band Together,"When Branch's brother, Floyd, is kidnapped for..."
4,726209,Leave the World Behind,A family's getaway to a luxurious rental home ...


In [72]:
ps =PorterStemmer()
def stem(text): # stem("dancing")=>"danc"
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)
data_model["label"]=data_model["label"].apply(stem)
data_model["label"]=data_model["label"].apply(lambda x : x.lower())
data_model.head()

,id,title,label
0,466420,Killers of the Flower Moon,when oil is discov in 1920 oklahoma under osag...
1,897087,Freelance,an ex-speci forc oper take a job to provid sec...
2,787699,Wonka,willi wonka – chock-ful of idea and determin t...
3,901362,Trolls Band Together,"when branch' brother, floyd, is kidnap for hi ..."
4,726209,Leave the World Behind,a family' getaway to a luxuri rental home take...


In [77]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')
vector = cv.fit_transform(data_model['label']).toarray()

In [78]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)
similarity

array([[1.        , 0.        , 0.        , 0.        , 0.05063697,
        0.01687899, 0.        , 0.03944297, 0.11026699, 0.01869894,
        0.01703748, 0.        , 0.01913898, 0.01657484, 0.07297564,
        0.        , 0.01809233, 0.        ],
       [0.        , 1.        , 0.02666904, 0.02023612, 0.        ,
        0.04474374, 0.        , 0.01742626, 0.        , 0.04956816,
        0.1806555 , 0.02324953, 0.02536731, 0.02196874, 0.03224129,
        0.04474374, 0.02398006, 0.0631474 ],
       [0.        , 0.02666904, 1.        , 0.11980846, 0.        ,
        0.08830216, 0.02782074, 0.01719544, 0.02060214, 0.        ,
        0.        , 0.02294157, 0.02503131, 0.086711  , 0.        ,
        0.06622662, 0.02366243, 0.0830813 ],
       [0.        , 0.02023612, 0.11980846, 1.        , 0.        ,
        0.13400504, 0.        , 0.02609534, 0.        , 0.        ,
        0.01690792, 0.        , 0.03798686, 0.08224396, 0.02414023,
        0.08375315, 0.03590948, 0.0945615 ],
    

In [88]:
similarity.shape

(18, 18)

In [89]:
def recommend(movie,m):
    index = data_model[data_model['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:m+1]:
        print(data_model.iloc[i[0]].title)

In [96]:
recommend("Killers of the Flower Moon",2)

Oppenheimer
Fast Charlie


In [98]:
import pickle
pickle.dump(data_model.to_dict(),open("movies_dict.pkl","wb"))
pickle.dump(similarity,open("similarity.pkl","wb"))